# Составление SQL запросов в ClickHouse 

<hr>

данные сервиса аренды жилья  

### 1. listings – информация о жилье, включая полные описания, характеристики и средние оценки в отзывах;  
id – идентификатор объявления  
review_scores_rating – оценка на основе отзывов  
reviews_per_month – число отзывов в месяц  
host_id – идентификатор хозяина  
room_type – тип жилья ('Private room')  
latitude – широта  
longitude – долгота  
price - цена  
cleaning_fee - стоимость уборки  
last_review - последние отзывы  
amenities - список удобств  


### 2. calendar_summary – информация о доступности и цене того или иного жилья по дням  
listing_id – идентификатор жилья (объявления)  
date – дата   
available – доступность жилья в данный день (t/f)  
price – цена за ночь  

### 3. reviews – отзывы  
listing_id –  идентификатор жилья   
id – id отзыва  
date – дата отзыва  
reviewer_id – id ревьюера (автора отзыва)  
reviewer_name – имя автора  
comments – сам отзыв  

<hr>

данные интернет магазина  

### 4. installs — содержит данные об установках приложения по дням.  
DeviceID — идентификатор устройства, на которое было установлено приложение;  
InstallationDate — дата установки приложения;  
InstallCost — цена установки приложения в рублях;  
Platform — платформа, на которой было установлено приложение (iOS/ Android);  
Source — источник установки приложения (магазин приложения/ рекламная система/ переход с сайта).  
events — содержит данные о том, как активно пользователи просматривают товары в приложении по дням.  

### 5. events — содержит данные о том, как активно пользователи просматривают товары в приложении по дням.  
DeviceID — идентификатор устройства, на котором используется приложение;  
AppPlatform — платформа, на которой используется приложение (iOS/ Android);  
EventDate — дата, за которую собрана статистика;  
events — количество просмотров всех товаров за этот день у этого DeviceID.  
checks — содержит данные о покупках пользователей в приложении по дням  

### 6. checks — содержит данные о покупках пользователей в приложении по дням  
UserID — идентификатор пользователя;  
Rub — суммарный чек пользователя на дату;  
BuyDate — дата, за которую собрана статистика.  
 devices – чтобы просмотры и установки можно было объединить с покупками  

### 7. devices – чтобы просмотры и установки можно было объединить с покупками   
DeviceID — идентификатор устройства;    
UserID — идентификатор пользователя.  

In [21]:
import pandas as pd
import pandahouse as ph

In [22]:
#объявляем параметры подключения
connection = dict(host='http://clickhouse.beslan.pro:8080',
                      database='default',
                      user='student', 
                      password='')

Сначала оставьте только те объявления, в которых оценка на основе отзывов выше среднего, а число отзывов в месяц составляет строго меньше трёх.  
Затем отсортируйте по убыванию две колонки: сначала по числу отзывов в месяц, потом по оценке.  
В качестве ответа укажите id объявления из первой строки.

review_scores_rating – оценка на основе отзывов
reviews_per_month – число отзывов в месяц
id – идентификатор объявления
Таблица – listings. Наивысшие показатели = сортировка по убыванию.

In [26]:
query = """
WITH (SELECT AVG(rsr) FROM (SELECT toFloat32OrZero(review_scores_rating) AS rsr FROM listings WHERE rsr > 0)) AS avg_rsr
SELECT
    id,
    toFloat32OrZero(review_scores_rating) AS review_scores_rating,
    reviews_per_month
FROM listings
WHERE
    review_scores_rating > avg_rsr
    AND reviews_per_month < 3
ORDER BY
    reviews_per_month DESC,
    review_scores_rating DESC
LIMIT 5
"""
ph.read_clickhouse(query=query, connection=connection)

,id,review_scores_rating,reviews_per_month
0,22476940,100.0,2.99
1,15947545,97.0,2.99
2,4149532,100.0,2.98
3,21390172,99.0,2.98
4,24145948,99.0,2.98


Посчитайте среднее расстояние до центра города и выведите идентификаторы объявлений о сдаче отдельных комнат, для которых расстояние оказалось меньше среднего.  
Результат отсортируйте по убыванию, тем самым выбрав комнату, которая является наиболее удаленной от центра, но при этом расположена ближе, чем остальные комнаты в среднем. 

id – идентификатор объявления  
host_id – идентификатор хозяина  
room_type – тип жилья ('Private room')  
latitude – широта  
longitude – долгота  
52.5200 с.ш., 13.4050 в.д – координаты центра Берлина  

В качестве ответа укажите идентификатор хозяина (host_id), сдающего данную комнату.

In [27]:
query = """
WITH (SELECT AVG(geoDistance(13.4050, 52.5200, toFloat64(longitude), toFloat64(latitude))) FROM listings WHERE room_type == 'Private room') AS avg_distance
SELECT
    host_id,
    geoDistance(13.4050, 52.5200, toFloat64(longitude), toFloat64(latitude)) AS distance
FROM listings
WHERE
    distance < avg_distance
    AND room_type == 'Private room'
ORDER BY distance DESC
LIMIT 5
"""
ph.read_clickhouse(query=query, connection=connection)

,host_id,distance
0,163065208,4608.981934
1,54900115,4608.205078
2,25835088,4608.166016
3,21346093,4607.219238
4,12258154,4606.446777


Представим, что вы планируете снять жилье в Берлине на 7 дней, используя более хитрые фильтры, чем предлагаются на сайте.

Отберите объявления из таблицы listings, которые:

- находятся на расстоянии от центра меньше среднего  
- обойдутся дешевле 100$ в день (price с учетом cleaning_fee, который добавляется к общей сумме за неделю, т.е его нужно делить на кол-во дней)  
- имеют последние отзывы (last_review), начиная с 1 сентября 2018 года  
- имеют WiFi в списке удобств (amenities)  

Отсортируйте полученные значения по убыванию review_scores_rating и в качестве ответа укажите host_id из первой строки. 

In [28]:
query = """
WITH (SELECT AVG(geoDistance(13.4050, 52.5200, toFloat64OrNull(longitude), toFloat64OrNull(latitude))) FROM listings) AS avg_distance,
    (toFloat64OrNull(replaceRegexpAll(price, '[$,]', '')) + toFloat64OrNull(replaceRegexpAll(cleaning_fee, '[$,]', '')) / 7 ) AS acceptable_price

SELECT
    id,
    host_id,
    position('Wifi' IN amenities) > 0 as wifi,
    acceptable_price,
    toDateOrZero(last_review) AS last_review,
    toFloat64OrNull(review_scores_rating) AS review_scores_rating,
    geoDistance(13.4050, 52.5200, toFloat64OrNull(longitude), toFloat64OrNull(latitude)) AS distance,
    avg_distance
FROM listings
WHERE
    distance < avg_distance
    AND acceptable_price < 100
    AND last_review >= '2018-09-01'
    AND wifi = 1
ORDER BY review_scores_rating DESC
LIMIT 5
"""
ph.read_clickhouse(query=query, connection=connection)

,id,host_id,wifi,acceptable_price,last_review,review_scores_rating,distance,avg_distance
0,28071741,39740677,1,53.571429,2018-11-04,100.0,2528.391602,4575.663526
1,15186943,20173692,1,81.428571,2018-09-16,100.0,4208.450195,4575.663526
2,7197260,19577506,1,52.857143,2018-09-02,100.0,3922.759277,4575.663526
3,3034573,9208115,1,52.857143,2018-10-21,100.0,2411.463379,4575.663526
4,28069928,46659004,1,23.000000,2018-11-06,100.0,3760.414551,4575.663526


Давайте найдем в таблице calendar_summary те доступные (available='t') объявления, у которых число отзывов от уникальных пользователей в таблице reviews выше среднего.

NB! Для простоты будем считать, что отзыв — это уникальный посетитель на уникальное жилье, не учитывая возможные повторные отзывы от того же посетителя.

Результат отфильтруйте так, чтобы остались только записи, у которых число отзывов от уникальных людей выше среднего.

Отсортируйте результат по возрастанию listing_id и в качестве ответа впишите количество отзывов от уникальных пользователей из первой строки.

In [31]:
query = """
WITH (
    SELECT
        AVG(review)
    FROM
        (
        SELECT uniqExact(reviewer_id) AS review
        FROM reviews
        GROUP BY listing_id
        )
    ) AS avg_review

SELECT
    r.listing_id,
    r.reviews
FROM
    (
    SELECT
        listing_id,
        COUNT(reviewer_id) AS reviews
    FROM reviews
    GROUP BY listing_id
    HAVING reviews > avg_review
    ) AS r

JOIN
    (
    SELECT DISTINCT(listing_id)
    FROM calendar_summary
    WHERE available = 't'
    ) AS cs
        ON cs.listing_id=r.listing_id

ORDER BY listing_id
LIMIT 5
"""
ph.read_clickhouse(query=query, connection=connection)

,listing_id,reviews
0,2015,118
1,3176,143
2,3309,25
3,7071,197
4,14325,23


Разделите всех покупателей на сегменты:

NB! Правые границы берутся не включительно, например, чек в 10 рублей будет относиться к сегменту С

А — средний чек покупателя менее 5 ₽  
B — средний чек покупателя от 5-10 ₽  
C — средний чек покупателя от 10-20 ₽  
D — средний чек покупателя от 20 ₽  

Отсортируйте результирующую таблицу по возрастанию UserID и укажите сегмент четвертого пользователя.

In [32]:
query = """
SELECT
    UserID,
    avg_rub,
    CASE
        WHEN avg_rub < 5 THEN 'A'
        WHEN avg_rub < 10 THEN 'B'
        WHEN avg_rub < 20 THEN 'C'
        ELSE 'D'
    END AS segment
FROM
    (
    SELECT
        AVG(Rub) AS avg_rub,
        UserID
    FROM checks
    GROUP BY UserID
    )
ORDER BY UserID
LIMIT 5
"""
ph.read_clickhouse(query=query, connection=connection)

,UserID,avg_rub,segment
0,1406814346437,3.5,A
1,3116519148596,5.0,B
2,3827093218449,5.5,B
3,4301734536835,10.0,C
4,9820728992879,1.0,A


Используйте предыдущий запрос как подзапрос и посчитайте, сколько клиентов приходится на каждый сегмент и сколько доходов он приносит.  
Отсортируйте результат по убыванию суммы доходов на сегмент и в качестве ответа укажите наибольшую сумму.

In [33]:
query = """
SELECT
    s.segment,
    SUM(c.Rub) AS Rub
FROM checks AS c

JOIN
    (
    SELECT
        UserID,
        CASE
            WHEN avg_rub < 5 THEN 'A'
            WHEN avg_rub < 10 THEN 'B'
            WHEN avg_rub < 20 THEN 'C'
            ELSE 'D'
        END AS segment
    FROM
        (
        SELECT
            AVG(Rub) AS avg_rub,
            UserID
        FROM checks
        GROUP BY UserID
        )
    ) AS s
        ON s.UserID=c.UserID
        
GROUP BY s.segment
ORDER BY Rub DESC
"""
ph.read_clickhouse(query=query, connection=connection)

,segment,Rub
0,D,250551286
1,C,223388252
2,B,72257465
3,A,65057410


Предположим, что в выборе жилья нас интересует только два параметра: наличие кухни (kitchen) и гибкой системы отмены (flexible), причем первый в приоритете.

Создайте с помощью оператора CASE колонку с обозначением группы, в которую попадает жилье из таблицы listings:

- 'good', если в удобствах (amenities) присутствует кухня и система отмены (cancellation_policy) гибкая
- 'ok', если в удобствах есть кухня, но система отмены не гибкая
- 'not ok' во всех остальных случаях

Результат отсортируйте по новой колонке по возрастанию, установите ограничение в 5 строк, в качестве ответа укажите host_id первой строки.

Обратите внимание, что cancellation_policy - это отдельная колонка, по ней необходимо смотреть систему отмены

In [34]:
query = """
SELECT
    host_id,
    CASE
        WHEN
            position('Kitchen' IN amenities) > 0
            AND cancellation_policy = 'flexible'
                THEN 'good'
        WHEN
            position('Kitchen' IN amenities) > 0
                THEN 'ok'
        ELSE 'not ok'
    END AS status
FROM listings
ORDER BY status
LIMIT 5
"""
ph.read_clickhouse(query=query, connection=connection)

,host_id,status
0,27607502,good
1,75398823,good
2,10780623,good
3,6637229,good
4,199462820,good
